Imports and helper functions

In [1]:
#Code source: https://github.com/ageron/handson-ml2/blob/master/02_end_to_end_machine_learning_project.ipynb

# Python ≥3.5 is required
import sys
assert sys.version_info >= (3, 5)

# Scikit-Learn ≥0.20 is required
import sklearn
assert sklearn.__version__ >= "0.20"

# Common imports
import numpy as np
import os

# To plot pretty figures
%matplotlib inline
import matplotlib as mpl
import matplotlib.pyplot as plt
mpl.rc('axes', labelsize=14)
mpl.rc('xtick', labelsize=12)
mpl.rc('ytick', labelsize=12)

# Where to save the figures
PROJECT_ROOT_DIR = "."
IMAGES_PATH = os.path.join(PROJECT_ROOT_DIR, "Report", "fig")
os.makedirs(IMAGES_PATH, exist_ok=True)

def save_fig(fig_id, tight_layout=True, fig_extension="png", resolution=300):
    path = os.path.join(IMAGES_PATH, fig_id + "." + fig_extension)
    print("Saving figure", fig_id)
    if tight_layout:
        plt.tight_layout()
    plt.savefig(path, format=fig_extension, dpi=resolution)

Regression MLP imports

In [2]:
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
from sklearn.neural_network import MLPRegressor
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import r2_score, max_error

RNN imports

In [3]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

""" import torch
from torch import nn
from torch.utils.data import Dataset, DataLoader
from tqdm import tqdm """

' import torch\nfrom torch import nn\nfrom torch.utils.data import Dataset, DataLoader\nfrom tqdm import tqdm '

Tensor csv reading method from "Hands-On Machine Learning..."

In [ ]:
def parse_csv_line(line):
    defs = [0.] * n_inputs + [tf.constant([], dtype=tf.float32)]
    fields = tf.io.decode_csv(line, record_defaults=defs)
    return tf.stack(fields[:-1]), tf.stack(fields[-1:])

def preprocess(line):
    x, y = parse_csv_line(line)
    return (x - X_mean) / X_std, y

In [ ]:
def csv_reader_dataset(filepaths, n_readers=5, n_read_threads=None,
                       n_parse_threads=5, shuffle_buffer_size=10_000, seed=42,
                       batch_size=32):
    dataset = tf.data.Dataset.list_files(filepaths, seed=seed)
    dataset = dataset.interleave(
        lambda filepath: tf.data.TextLineDataset(filepath).skip(1),
        cycle_length=n_readers, num_parallel_calls=n_read_threads)
    dataset = dataset.map(preprocess, num_parallel_calls=n_parse_threads)
    dataset = dataset.shuffle(shuffle_buffer_size, seed=seed)
    return dataset.batch(batch_size).prefetch(1)

Function for making models and generating plots, stolen from previous solution

In [4]:
def test_model(X_train, y_train, X_test, y_test, model):
 
    y_train_predicted = model.predict(X_train)
    y_test_predicted = model.predict(X_test)
    
    print('--------------------TRAIN SET-------------------------')
    print('r2_score = ','%.2f' % r2_score(y_train,y_train_predicted), '(1.0 means perfect fit)')
    print('max_error = ','%.2f' % max_error(y_train,y_train_predicted))
    print('root_mean_squared_error = ','%.2f' % mean_squared_error(y_train,y_train_predicted, squared=False))
    print('--------------------TEST SET-------------------------')
    print('r2_score = ','%.2f' % r2_score(y_test,y_test_predicted), '(1.0 means perfect fit)')
    print('max_error = ','%.2f' % max_error(y_test,y_test_predicted))
    print('root_mean_squared_error = ','%.2f' % mean_squared_error(y_test,y_test_predicted, squared=False))
    print(y_train.index[:100])
    
    return r2_score(y_train,y_train_predicted), r2_score(y_test,y_test_predicted)

Import the training set and the data set

In [5]:
import pandas as pd
useless_features = ['engine','RUL', 'cycle', 'setting 3', 'sensor 1', 'sensor 5', 'sensor 6', 'sensor 10','sensor 14', 'sensor 16', 'sensor 18', 'sensor 19']

fd_001_train  = pd.read_csv('train_FD001.csv')
y_train_full = fd_001_train['RUL']
X_train_full = fd_001_train.drop(columns=useless_features)

fd_001_test = pd.read_csv('test_FD001.csv')
y_test = fd_001_test['RUL']
X_test = fd_001_test.drop(columns=useless_features)

""" X_mlp_train_full = X_train.copy()
y_mlp_train_full = y_train.copy()
X_mlp_test = X_train.copy()
y_mlp_test = y_train.copy() """

""" X_mlp_train, X_mlp_valid, y_mlp_train, y_mlp_valid = train_test_split(
    X_mlp_train_full, y_mlp_train_full, random_state=42) """

""" fan_prepared_test = num_pipeline.fit_transform(fd_001_test)
fan_labels_test = fd_001_test["RUL"].copy() """

' fan_prepared_test = num_pipeline.fit_transform(fd_001_test)\nfan_labels_test = fd_001_test["RUL"].copy() '

In [6]:
X_train_full.describe()

,setting 1,setting 2,sensor 2,sensor 3,sensor 4,sensor 7,sensor 8,sensor 9,sensor 11,sensor 12,sensor 13,sensor 15,sensor 17,sensor 20,sensor 21
count,20631.000000,20631.000000,20631.000000,20631.000000,20631.000000,20631.000000,20631.000000,20631.000000,20631.000000,20631.000000,20631.000000,20631.000000,20631.000000,20631.000000,20631.000000
mean,-0.000009,0.000002,642.680934,1590.523119,1408.933782,553.367711,2388.096652,9065.242941,47.541168,521.413470,2388.096152,8.442146,393.210654,38.816271,23.289705
std,0.002187,0.000293,0.500053,6.131150,9.000605,0.885092,0.070985,22.082880,0.267087,0.737553,0.071919,0.037505,1.548763,0.180746,0.108251
min,-0.008700,-0.000600,641.210000,1571.040000,1382.250000,549.850000,2387.900000,9021.730000,46.850000,518.690000,2387.880000,8.324900,388.000000,38.140000,22.894200
25%,-0.001500,-0.000200,642.325000,1586.260000,1402.360000,552.810000,2388.050000,9053.100000,47.350000,520.960000,2388.040000,8.414900,392.000000,38.700000,23.221800
50%,0.000000,0.000000,642.640000,1590.100000,1408.040000,553.440000,2388.090000,9060.660000,47.510000,521.480000,2388.090000,8.438900,393.000000,38.830000,23.297900
75%,0.001500,0.000300,643.000000,1594.380000,1414.555000,554.010000,2388.140000,9069.420000,47.700000,521.950000,2388.140000,8.465600,394.000000,38.950000,23.366800
max,0.008700,0.000600,644.530000,1616.910000,1441.490000,556.060000,2388.560000,9244.590000,48.530000,523.380000,2388.560000,8.584800,400.000000,39.430000,23.618400


<h2>Multi-layer Perceptron</h2>

3 hidden layers, Adam optimizer

In [7]:
mlp_reg = MLPRegressor(hidden_layer_sizes=[50, 50, 50], random_state=42)
pipeline_mlp_1 = make_pipeline(StandardScaler(), mlp_reg)
pipeline_mlp_1.fit(X_train_full, y_train_full)
y_pred = pipeline_mlp_1.predict(X_test)
rmse = mean_squared_error(y_test, y_pred, squared=False)

In [8]:
rmse

46.594817064172354

3 hidden layers, lbfgs optimizer

In [9]:
mlp_reg = MLPRegressor(hidden_layer_sizes=[50, 50, 50], random_state=42, solver='lbfgs')
pipeline_mlp_2 = make_pipeline(StandardScaler(), mlp_reg)
pipeline_mlp_2.fit(X_train_full, y_train_full)
y_pred = pipeline_mlp_2.predict(X_test)
rmse = mean_squared_error(y_test, y_pred, squared=False)

c:\Program Files\Python310\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:559: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
  self.n_iter_ = _check_optimize_result("lbfgs", opt_res, self.max_iter)


In [10]:
rmse

47.47616185711904

Adam optimizer with lower learning rate

In [11]:
mlp_reg = MLPRegressor(hidden_layer_sizes=[50, 50, 50], random_state=42, learning_rate_init=0.0005, max_iter=600)
pipeline_mlp_3 = make_pipeline(StandardScaler(), mlp_reg)
pipeline_mlp_3.fit(X_train_full, y_train_full)
y_pred = pipeline_mlp_3.predict(X_test)
rmse = mean_squared_error(y_test, y_pred, squared=False)

In [12]:
test_model(X_train_full,y_train_full,X_test,y_test, pipeline_mlp_1)
test_model(X_train_full,y_train_full,X_test,y_test, pipeline_mlp_2)
test_model(X_train_full,y_train_full,X_test,y_test, pipeline_mlp_3)

--------------------TRAIN SET-------------------------
r2_score =  0.65 (1.0 means perfect fit)
max_error =  223.69
root_mean_squared_error =  40.63
--------------------TEST SET-------------------------
r2_score =  0.38 (1.0 means perfect fit)
max_error =  218.75
root_mean_squared_error =  46.59
RangeIndex(start=0, stop=100, step=1)
--------------------TRAIN SET-------------------------
r2_score =  0.67 (1.0 means perfect fit)
max_error =  211.51
root_mean_squared_error =  39.67
--------------------TEST SET-------------------------
r2_score =  0.35 (1.0 means perfect fit)
max_error =  227.81
root_mean_squared_error =  47.48
RangeIndex(start=0, stop=100, step=1)
--------------------TRAIN SET-------------------------
r2_score =  0.66 (1.0 means perfect fit)
max_error =  217.51
root_mean_squared_error =  40.05
--------------------TEST SET-------------------------
r2_score =  0.38 (1.0 means perfect fit)
max_error =  214.91
root_mean_squared_error =  46.47
RangeIndex(start=0, stop=100, ste

(0.6619856937750027, 0.379175645112903)

<h2>Recurrent Neural Network</h2>

Let's try with tensor

In [19]:
model_rnn = keras.models.Sequential()
model_rnn.add(keras.Input(shape=(15,50)))
model_rnn.add(layers.SimpleRNN(50, return_sequences=True, activation='relu'))
model_rnn.add(layers.SimpleRNN(50, return_sequences=False, activation='relu'))
#model_rnn.add(layers.SimpleRNN(50, return_sequences=True, activation='relu'))
#model_rnn.add(layers.LSTM(50, return_sequences=False, activation='relu'))
model_rnn.add(layers.Dense(1))
print(model_rnn.summary())

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 simple_rnn_3 (SimpleRNN)    (None, 15, 50)            5050      
                                                                 
 simple_rnn_4 (SimpleRNN)    (None, 50)                5050      
                                                                 
 dense_1 (Dense)             (None, 1)                 51        
                                                                 
Total params: 10,151
Trainable params: 10,151
Non-trainable params: 0
_________________________________________________________________
None


In [14]:
loss = keras.losses.SparseCategoricalCrossentropy(from_logits=True)
optim = keras.optimizers.Adam(lr=0.001)
metrics = ["accuracy"]

c:\Program Files\Python310\lib\site-packages\keras\optimizers\optimizer_v2\adam.py:114: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super().__init__(name, **kwargs)


In [15]:
model_rnn.compile(loss=loss, optimizer=optim, metrics=metrics)

In [17]:
dataset = tf.data.Dataset

TypeError: 'tuple' object is not callable

In [16]:
# training
batch_size = 64
epochs = 5

model_rnn.fit(X_train_full, y_train_full, batch_size=batch_size, epochs=epochs, verbose=2)

# evaulate
model_rnn.evaluate(X_test, y_test, batch_size=batch_size, verbose=2)

Epoch 1/5


ValueError: in user code:

    File "c:\Program Files\Python310\lib\site-packages\keras\engine\training.py", line 1160, in train_function  *
        return step_function(self, iterator)
    File "c:\Program Files\Python310\lib\site-packages\keras\engine\training.py", line 1146, in step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    File "c:\Program Files\Python310\lib\site-packages\keras\engine\training.py", line 1135, in run_step  **
        outputs = model.train_step(data)
    File "c:\Program Files\Python310\lib\site-packages\keras\engine\training.py", line 993, in train_step
        y_pred = self(x, training=True)
    File "c:\Program Files\Python310\lib\site-packages\keras\utils\traceback_utils.py", line 70, in error_handler
        raise e.with_traceback(filtered_tb) from None
    File "c:\Program Files\Python310\lib\site-packages\keras\engine\input_spec.py", line 232, in assert_input_compatibility
        raise ValueError(

    ValueErro